[![Fixel Algorithms](https://i.imgur.com/AqKHVZ0.png)](https://fixelalgorithms.gitlab.io/)

# AI Program

## Machine Learning - Deep Learning - Object Detection

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 16/06/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0099DeepLearningObjectDetection.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import ParameterGrid

# Deep Learning
import torch
import torch.nn            as nn
import torch.nn.functional as F
from torch.optim.optimizer import Optimizer
from torch.optim.lr_scheduler import LRScheduler
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchinfo
from torchmetrics.classification import MulticlassAccuracy
from torchmetrics.detection.iou import IntersectionOverUnion

import torchvision
from torchvision.transforms import v2 as TorchVisionTrns

# Miscellaneous
import copy
from enum import auto, Enum, unique
import math
import os
from platform import python_version
import random
import shutil
import time

# Typing
from typing import Callable, Dict, Generator, List, Optional, Self, Set, Tuple, Union

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import HTML, Image
from IPython.display import display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout, SelectionSlider
from ipywidgets import interact

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

 ```python
 vallToFill = ???
 ```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

???
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

# Improve performance by benchmarking
torch.backends.cudnn.benchmark = True

# Reproducibility (Per PyTorch Version on the same device)
# torch.manual_seed(seedNum)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark     = False #<! Makes things slower


In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2

DATA_SET_FILE_NAME      = 'archive.zip'
DATA_SET_FOLDER_NAME    = 'IntelImgCls'

D_CLASSES  = {0: 'Red', 1: 'Green', 2: 'Blue'}
L_CLASSES  = ['R', 'G', 'B']
T_IMG_SIZE = (100, 100, 3)

DATA_FOLDER_PATH    = 'Data'
TENSOR_BOARD_BASE   = 'TB'


In [ ]:
# Download Auxiliary Modules for Google Colab
if runInGoogleColab:
    !wget https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/master/AIProgram/2024_02/DataManipulation.py
    !wget https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/master/AIProgram/2024_02/DataVisualization.py
    !wget https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/master/AIProgram/2024_02/DeepLearningPyTorch.py

In [ ]:
# Courses Packages

from DataManipulation import BBoxFormat
from DataManipulation import GenLabeldEllipseImg
from DataVisualization import PlotBox, PlotBBox, PlotLabelsHistogram
from DeepLearningPyTorch import ObjectDetectionDataset, ToTensor, YoloGrid
from DeepLearningPyTorch import GenDataLoaders, InitWeightsKaiNorm, TrainModel, TrainModelSch


* <font color='blue'>(**!**)</font> Go through `GenLabeldDataEllipse()`.
* <font color='blue'>(**!**)</font> Go through `ObjectDetectionDataset`.
* <font color='blue'>(**!**)</font> Go through `YoloGrid`.

In [ ]:
# General Auxiliary Functions

def GenData( numSamples: int, tuImgSize: Tuple[int, int, int], maxObj: int, boxFormat: BBoxFormat = BBoxFormat.YOLO ) -> Tuple[np.ndarray, List[np.ndarray], List[np.ndarray]]:

    mX = np.empty(shape = (numSamples, *tuImgSize[::-1]))
    lY = [None] * numSamples
    lB = [None] * numSamples

    for ii in range(numSamples):
        numObj = np.random.randint(maxObj) + 1
        mI, vLbl, mBB = GenLabeldEllipseImg(tuImgSize[:2], numObj, boxFormat = boxFormat)
        mX[ii]  = np.transpose(mI, (2, 0, 1))
        lY[ii]  = vLbl
        lB[ii]  = mBB
    
    return mX, lY, lB
        
# Data Loader
# Using a function to mitigate Multi Process issues:
# https://pytorch.org/docs/stable/notes/windows.html#multiprocessing-error-without-if-clause-protection
def DataLoaderBatch( dlData: DataLoader ) -> Tuple:
    
    return next(iter(dlData)) #<! PyTorch Tensors


## Object Detection

The _Object Detection_ task generalizes the _Object Localization_ task in 2 manners:

1. Support for many objects at the same image.
2. Detection as if there is any object at all.

This notebook demonstrates:
 - Generating a synthetic data set.
 - Generating the _target_ data in the YOLO form.
 - Building a model for _Object Detection_.
 - Training a model with a composed objective.

</br>

* <font color='brown'>(**#**)</font> The _Object Detection_ in this notebook is applies in _YOLO_ style: Single Pass, Grid and Anchors.
* <font color='brown'>(**#**)</font> The motivation for a synthetic dataset is being able to implement the whole training process (Existing datasets are huge).  
  Yet the ability to create synthetic dataset is a useful skill.
* <font color='brown'>(**#**)</font> There are known datasets for object detection: [COCO Dataset](https://cocodataset.org), [PASCAL VOC](http://host.robots.ox.ac.uk/pascal/VOC/).   
  They also define standards for the labeling system.  
  Training them is on the scale of days.
* <font color='brown'>(**#**)</font> [Object Detection Annotation Formats](https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation).
* <font color='brown'>(**#**)</font> Review of Object Detection approaches is given by Lilian Weng: [Part 1: Gradient Vector, HOG, and SS](https://lilianweng.github.io/posts/2017-10-29-object-recognition-part-1), [Part 2: CNN, DPM and Overfeat](https://lilianweng.github.io/posts/2017-12-15-object-recognition-part-2), [Part 3: R-CNN Family](https://lilianweng.github.io/posts/2017-12-31-object-recognition-part-3), [Part 4: Fast Detection Models](https://lilianweng.github.io/posts/2018-12-27-object-recognition-part-4).

In [ ]:
# Parameters

# Data
numSamplesTrain = 30_000
numSamplesVal   = 10_000
boxFormat       = BBoxFormat.YOLO
numCls          = len(L_CLASSES) #<! Number of classes
maxObj          = 3

# Model
gridSize = 5 #<! The gris is (gridSize x gridSize) 

# Training
batchSize   = 256
numWorkers  = 2 #<! Number of workers
numEpochs   = 35
λ = 20.0 #<! Localization Loss
ϵ = 0.1 #<! Label Smoothing

# Visualization
numImg = 3


## Generate / Load Data

The data is synthetic data.  
Each image includes Ellipses where its color is the class (`R`, `G`, `B`) and the bounding rectangle.

* <font color='brown'>(**#**)</font> The label, per object, is a vector of `5`: `[Class, xCenter, yCenter, boxWidth, boxHeight]`.  
* <font color='brown'>(**#**)</font> The label is in `YOLO` format, hence it is normalized to `[0, 1]`.


In [ ]:
# Image Sample

mI, vY, mBB = GenLabeldEllipseImg(T_IMG_SIZE[:2], maxObj, boxFormat = boxFormat)
vYY = np.array([L_CLASSES[clsIdx] for clsIdx in vY])
hA = PlotBox(mI, vYY, mBB)

* <font color='brown'>(**#**)</font> One could use negative values for the bounding box. The model will extrapolate the object dimensions.

In [ ]:
# Generate Data

mXTrain, lYTrain, lBBTrain = GenData(numSamplesTrain, T_IMG_SIZE, maxObj, boxFormat = boxFormat)
mXVal,   lYVal,   lBBVal   = GenData(numSamplesVal, T_IMG_SIZE, maxObj, boxFormat = boxFormat)

print(f'The training data set data shape: {mXTrain.shape}')
print(f'The training data set labels length: {len(lYTrain)}')
print(f'The training data set box length: {len(lBBTrain)}')
print(f'The validation data set data shape: {mXVal.shape}')
print(f'The validation data set labels length: {len(lYVal)}')
print(f'The validation data set box length: {len(lBBVal)}')

* <font color='red'>(**?**)</font> Why are lists used instead of arrays for the labels and the bounding boxes?

In [ ]:
# Generate Data

dsTrain = ObjectDetectionDataset(mXTrain, lYTrain, lBBTrain)
dsVal   = ObjectDetectionDataset(mXVal, lYVal, lBBVal)
lClass  = list(np.r_[*dsTrain.lY])


print(f'The training data set data shape: {dsTrain.tX.shape}')
print(f'The test data set data shape: {dsVal.tX.shape}')
print(f'The unique values of the labels: {np.unique(lClass)}')

* <font color='brown'>(**#**)</font> PyTorch with the `v2` transforms deals with bounding boxes using special type: `BoundingBoxes`.
* <font color='brown'>(**#**)</font> For _data augmentation_ see:
    - [Transforming and Augmenting Images](https://pytorch.org/vision/stable/transforms.html).
    - [Getting Started with Transforms v2](https://pytorch.org/vision/stable/auto_examples/transforms/plot_transforms_getting_started.html).
    - [Transforms v2: End to End Object Detection / Segmentation Example](https://pytorch.org/vision/stable/auto_examples/transforms/plot_transforms_e2e.html).
    - [How to Write Your Own v2 Transforms](https://pytorch.org/vision/stable/auto_examples/transforms/plot_custom_transforms.html).

In [ ]:
mX, mY = dsTrain[0]

In [ ]:
# Element of the Data Set

mX, mY = dsTrain[2]

vY = mY[:, 0].astype(np.int_)
mB = mY[:, 1:]

print(f'The features shape: {mX.shape}')
print(f'The label shape: {vY.shape}')
print(f'The bounding box shape: {mB.shape}')

* <font color='brown'>(**#**)</font> Since the labels are in the same contiguous container as the bounding box parameters, their type is `Float`.
* <font color='brown'>(**#**)</font> The bounding box is using absolute values. In practice it is commonly normalized to the image dimensions.

### Plot the Data

In [ ]:
# Plot the Data

vYY = np.array([L_CLASSES[clsIdx] for clsIdx in vY])
hA = PlotBox(np.transpose(mX, (1, 2, 0)), vYY, mB)


In [ ]:
# Histogram of Labels

hA = PlotLabelsHistogram(np.r_[*dsTrain.lY], lClass = L_CLASSES)
plt.show()

* <font color='red'>(**?**)</font> Explain the amount of samples in the histogram per class and in total.

### YOLO Style Transformer

The target data should be altered into a YOLO grid style.  
Namely the bounding box should be redefined relative to the grid cell the center of the box reside at.

![](https://i.imgur.com/CE1Ef7g.png)

This section implements the transformation as a PyTorch transformation.

In [ ]:
# Transform into YOLO Grid
# 1. Convert to a Torch tensor.
# 2. Transform int YOLO Grid format.

oTrns = TorchVisionTrns.Compose([
    ToTensor(),
    YoloGrid(gridSize),
])

In [ ]:
# Redefine the Data Set

dsTrain = ObjectDetectionDataset(mXTrain, lYTrain, lBBTrain, hDataTrans = oTrns)
dsVal   = ObjectDetectionDataset(mXVal, lYVal, lBBVal, hDataTrans = oTrns)

In [ ]:
# Extract Data

mX, mY = dsTrain[10]

In [ ]:
vY = np.array(mY[:, -1]).astype(np.int_)
vY.dtype

In [ ]:
vY.shape

In [ ]:
vYY = np.array([L_CLASSES[clsIdx] for clsIdx in vY])

In [ ]:
vY = np.array(mY[:, -1]).astype(np.int_)
mB = mY[:, 1:-1]
vYY = np.array([L_CLASSES[clsIdx] for clsIdx in vY])
hA = PlotBox(np.transpose(mX, (1, 2, 0)), vYY, mB)
hA.grid(True)

### Data Loaders

This section defines the data loaded.



In [ ]:
# Data Loader

dlTrain = torch.utils.data.DataLoader(dsTrain, shuffle = True, batch_size = 1 * batchSize, num_workers = numWorkers, drop_last = True, persistent_workers = True)
dlVal   = torch.utils.data.DataLoader(dsVal, shuffle = False, batch_size = 2 * batchSize, num_workers = numWorkers, persistent_workers = True)

# dlTrain = torch.utils.data.DataLoader(dsTrain, shuffle = True, batch_size = 1 * batchSize, num_workers = 0, drop_last = True, persistent_workers = False)
# dlVal   = torch.utils.data.DataLoader(dsVal, shuffle = False, batch_size = 2 * batchSize, num_workers = 0, persistent_workers = False)


In [ ]:
# Iterate on the Loader
# The first batch.

tX, mY = DataLoaderBatch(dlTrain)

print(f'The batch features dimensions: {tX.shape}')
print(f'The batch labels dimensions: {mY[:, 0].shape}')
print(f'The batch bounding box dimensions: {mY[:, 1:].shape}')

## The Model

This section defines the model.  

![](https://i.imgur.com/8oqg3Du.png)  
**Credit**: Optimized Convolutional Neural Network Architectures for Efficient On Device Vision based Object Detection.

![](https://i.imgur.com/AGQaauN.png)  
**Credit**: [Farid at @ai_fast_track](https://x.com/ai_fast_track/status/1453368771285032971)

* <font color='brown'>(**#**)</font> The following implementation has a model with a single output, both for the regression and the classification.
* <font color='brown'>(**#**)</font> One could create 2 different outputs (_Heads_) for each task.
* <font color='brown'>(**#**)</font> [Finally understand Anchor Boxes in Object Detection (2D and 3D)](https://www.thinkautonomous.ai/blog/anchor-boxes/) - How the prior is set by clustering the data.
* <font color='brown'>(**#**)</font> Anchor Free Model: [Forget the Hassles of Anchor Boxes with FCOS: Fully Convolutional One Stage Object Detection](https://scribe.rip/fc0e25622e1c).

In [ ]:
# Model
# Model generating function.

def BuildModel( numCls: int ) -> nn.Module:

    oModel = nn.Sequential(
        nn.Identity(),
        nn.Conv2d(3,   32,  3, stride = 2, padding = 0, bias = False), nn.BatchNorm2d(32 ), nn.ReLU(),
        nn.Conv2d(32,  32,  3, stride = 1, padding = 1, bias = False), nn.BatchNorm2d(32 ), nn.ReLU(),
        nn.Conv2d(32,  32,  3, stride = 2, padding = 0, bias = False), nn.BatchNorm2d(32 ), nn.ReLU(),
        nn.Conv2d(32,  32,  3, stride = 1, padding = 1, bias = False), nn.BatchNorm2d(32 ), nn.ReLU(),
        nn.Conv2d(32,  32,  3, stride = 1, padding = 1, bias = False), nn.BatchNorm2d(32 ), nn.ReLU(),
        nn.Conv2d(32,  64,  3, stride = 2, padding = 1, bias = False), nn.BatchNorm2d(64 ), nn.ReLU(),
        nn.Conv2d(64,  64,  3, stride = 1, padding = 1, bias = False), nn.BatchNorm2d(64 ), nn.ReLU(),
        nn.Conv2d(64,  64,  3, stride = 1, padding = 1, bias = False), nn.BatchNorm2d(64 ), nn.ReLU(),
        nn.Conv2d(64,  64,  3, stride = 1, padding = 1, bias = False), nn.BatchNorm2d(64 ), nn.ReLU(),
        nn.Conv2d(64,  64,  3, stride = 1, padding = 1, bias = False), nn.BatchNorm2d(64 ), nn.ReLU(),
        nn.Conv2d(64,  64,  3, stride = 2, padding = 1, bias = False), nn.BatchNorm2d(64 ), nn.ReLU(),
        nn.Conv2d(64,  128, 3, stride = 1, padding = 0, bias = False), nn.BatchNorm2d(128), nn.ReLU(),
        nn.Conv2d(128, 256, 3, stride = 1, padding = 0, bias = False), nn.BatchNorm2d(256), nn.ReLU(),
        nn.Conv2d(256, 512, 2, stride = 1, padding = 0, bias = False), nn.BatchNorm2d(512), nn.ReLU(),
        nn.Conv2d(512, numCls + 4, 1, stride = 1, padding = 0, bias = True),
        nn.Flatten()
    )

    return oModel 

* <font color='red'>(**?**)</font> What's the motivation for the depth of the model (Relatively deep)?
* <font color='red'>(**?**)</font> Explain the actual operation of the last `Conv2D` layer. Can it be replaced with a `Linear` layer?
* <font color='brown'>(**#**)</font> One could set the image to a power of 2. Then all convolution layers could have been with padding and stride of 2 until the size is `1x1`.

In [ ]:
# Build the Model

oModel = BuildModel(len(L_CLASSES))

In [ ]:
# Model Information
# Pay attention to the layers name.
torchinfo.summary(oModel, (batchSize, *(T_IMG_SIZE[::-1])), col_names = ['kernel_size', 'output_size', 'num_params'], device = 'cpu', row_settings = ['depth', 'var_names'])

* <font color='red'>(**?**)</font> Explain the dimensions of the last layer.
* <font color='red'>(**?**)</font> Will the model work with smaller images?

## Train the Model

This section trains the model.  

* <font color='brown'>(**#**)</font> The training loop must be adapted to the new loss function.

### Image Localization Loss

The loss is a composite of 2 loss functions:

$$\ell\left(\hat{\boldsymbol{y}},\boldsymbol{y}\right)=\lambda_{\text{MSE}}\cdot\ell_{\text{MSE}}\left(\hat{\boldsymbol{y}}_{\text{bbox}},\boldsymbol{y}_{\text{bbox}}\right)+\lambda_{\text{CE}}\cdot\ell_{\text{CE}}\left(\hat{\boldsymbol{y}}_{\text{label}},\boldsymbol{y}_{\text{label}}\right)$$

Where $\lambda_{\text{MSE}}$ and $\lambda_{\text{CE}}$ are the weights of each loss.

* <font color='brown'>(**#**)</font> In practice a single $\lambda$ is required.
* <font color='brown'>(**#**)</font> The MSE is not optimal loss function. It will be replaced by the _Log Euclidean_ loss.

In [ ]:
# Object Localization Loss
class ObjLocLoss( nn.Module ):
    def __init__( self, numCls: int, λ: float, ϵ: float = 0.0 ) -> None:
        super(ObjLocLoss, self).__init__()

        self.numCls     = numCls
        self.λ          = λ
        self.ϵ          = ϵ
        self.oMseLoss   = nn.MSELoss()
        self.oCeLoss    = nn.CrossEntropyLoss(label_smoothing = ϵ)
    
    def forward( self: Self, mYHat: torch.Tensor, mY: torch.Tensor ) -> torch.Tensor:

        mseLoss = self.oMseLoss(mYHat[:, self.numCls:], mY[:, 1:])
        ceLoss  = self.oCeLoss(mYHat[:, :self.numCls], mY[:, 0].to(torch.long))

        lossVal = (self.λ * mseLoss) + ceLoss
		
        return lossVal

### Image Localization Score

The score is defined by the _IoU_ of a valid classification:

$$\text{Score}=\frac{1}{N}\sum_{i=1}^{N}\mathbb{I}\left\{ \hat{y}_{i}=y_{i}\right\} \cdot\text{IoU}\left(\hat{B}_{i},B_{i}\right)$$

Where:
- $\hat{y}_{i}$ is the predicted label
- $y_{i}$ is the correct label
- $\hat{B}_{i}$ is the predicted bounding box
- $B_{i}$ is the correct bounding box
In other words, the average IoU, considering only correct (label) prediction.

* <font color='red'>(**?**)</font> What are the bounds of the values of the score function?
* <font color='red'>(**?**)</font> Is higher or lower value bette for the score?

In [ ]:
# Object Localization Score
class ObjLocScore( nn.Module ):
    def __init__( self, numCls: int ) -> None:
        super(ObjLocScore, self).__init__()

        self.numCls = numCls
    
    def forward( self: Self, mYHat: torch.Tensor, mY: torch.Tensor ) -> Tuple[float, float, float]:

        batchSize = mYHat.shape[0]
        
        vY, mBox = mY[:, 0].to(torch.long), mY[:, 1:]

        vIoU = torch.diag(torchvision.ops.box_iou(torchvision.ops.box_convert(mYHat[:, self.numCls:], 'cxcywh', 'xyxy'), torchvision.ops.box_convert(mBox, 'cxcywh', 'xyxy')))
        vCor = (vY == torch.argmax(mYHat[:, :self.numCls], dim = 1)).to(torch.float32) #<! Correct labels

        # valIoU      = torch.mean(vIoU).item()
        # valAcc      = torch.mean(vCor).item()
        valScore    = torch.inner(vIoU, vCor) / batchSize
		
        return valScore

In [ ]:
# Run Device

runDevice = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') #<! The 1st CUDA device


In [ ]:
# Loss and Score Function

hL = ObjLocLoss(numCls = numCls, λ = λ, ϵ = ϵ)
hS = ObjLocScore(numCls = numCls)

hL = hL.to(runDevice)
hS = hS.to(runDevice)


In [ ]:
# Training Loop

oModel = oModel.to(runDevice)
oOpt = torch.optim.AdamW(oModel.parameters(), lr = 1e-5, betas = (0.9, 0.99), weight_decay = 1e-5) #<! Define optimizer
oSch = torch.optim.lr_scheduler.OneCycleLR(oOpt, max_lr = 5e-4, total_steps = numEpochs)
_, lTrainLoss, lTrainScore, lValLoss, lValScore, lLearnRate = TrainModel(oModel, dlTrain, dlVal, oOpt, numEpochs, hL, hS, oSch = oSch)

In [ ]:
# Plot Training Phase

hF, vHa = plt.subplots(nrows = 1, ncols = 3, figsize = (12, 5))
vHa = np.ravel(vHa)

hA = vHa[0]
hA.plot(lTrainLoss, lw = 2, label = 'Train')
hA.plot(lValLoss, lw = 2, label = 'Validation')
hA.set_title(f'Object Localization Loss (λ = {λ:0.1f})')
hA.set_xlabel('Epoch')
hA.set_ylabel('Loss')
hA.legend()

hA = vHa[1]
hA.plot(lTrainScore, lw = 2, label = 'Train')
hA.plot(lValScore, lw = 2, label = 'Validation')
hA.set_title('Object Localization Score')
hA.set_xlabel('Epoch')
hA.set_ylabel('Score')
hA.legend()

hA = vHa[2]
hA.plot(lLearnRate, lw = 2)
hA.set_title('Learn Rate Scheduler')
hA.set_xlabel('Epoch')
hA.set_ylabel('Learn Rate')

In [ ]:
# Plot Prediction
# TODO: Check classification

rndIdx = np.random.randint(numSamplesVal)

mX, vY = dsVal[rndIdx]
valY    = int(vY[0])
vB      = vY[1:]
with torch.no_grad():
    tX = torch.tensor(mX)
    tX = torch.unsqueeze(tX, 0)
    tX = tX.to(runDevice)
    mYHat = oModel(tX).detach().cpu().numpy()

vYHat       = mYHat[0]
valYHat     = np.argmax(vYHat[:numCls])
vBHat       = vYHat[numCls:]

hA = PlotBox(np.transpose(mX, (1, 2, 0)), L_CLASSES[valY], vB)
hA = PlotBBox(hA, L_CLASSES[valYHat], vBHat)

* <font color='red'>(**?**)</font> What would be the results if teh generated data had more small ellipses?
* <font color='green'>(**@**)</font> Display the _accuracy_ and _IoU_ scores and _MSE_ and _CE_ loss over the epochs.   
  It will require updating the Loss, Score classes and the training function.